In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import yaml
import numpy as np
import pandas as pd

In [2]:
# Target web page:
url = "https://generalassemb.ly/locations/atlanta"

# Establishing the connection to the web page:
response = requests.get(url)

# You can use status codes to understand how the target server responds to your request.
# Ex., 200 = OK, 400 = Bad Request, 403 = Forbidden, 404 = Not Found.
print(response.status_code)
time.sleep(2)

200


In [3]:
html = response.text

In [4]:
# print(html)

In [5]:
soup = BeautifulSoup(html, "lxml")


In [6]:
soup.find('a', {'class': "LocationMasthead__contact_phone___3ntPh"}).text

'+1 (404) 334-7858'

In [7]:
soup.find('a', {'class': "LocationMasthead__contact_email___33lV7"}).text

'atlanta@generalassemb.ly'

In [8]:
soup.find('address', {'class': 'Locations__campus_address___2iaHf'}).text

'675 Ponce De Leon Avenue NE, 2nd FloorAtlanta, GA 30308'

In [9]:
soup.find_all('p', {'class':'ProgramPanel__topic___3Jtm0'})

[<p class="ProgramPanel__topic___3Jtm0" data-reactid="227">Coding</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="233">Data</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="239">Design</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="253">Business</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="259">Coding</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="265">Coding</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="271">Coding</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="277">Coding</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="283">Data</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="289">Data</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="295">Design</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="301">Design</p>,
 <p class="ProgramPanel__topic___3Jtm0" data-reactid="307">Marketing</p>]

In [10]:
soup.find('p', {'class':'ProgramPanel__topic___3Jtm0'}).text

'Coding'

In [11]:
soup.find_all('a', {'class': 'ProgramPanel__title___13Fzd'})

[<a class="ProgramPanel__title___13Fzd" data-reactid="228" href="/education/software-engineering-immersive?where=atlanta">Software Engineering Immersive</a>,
 <a class="ProgramPanel__title___13Fzd" data-reactid="234" href="/education/data-science-immersive?where=atlanta">Data Science Immersive</a>,
 <a class="ProgramPanel__title___13Fzd" data-reactid="240" href="/education/user-experience-design-immersive?where=atlanta">User Experience Design Immersive</a>,
 <a class="ProgramPanel__title___13Fzd" data-reactid="254" href="/education/product-management?where=atlanta">Product Management</a>,
 <a class="ProgramPanel__title___13Fzd" data-reactid="260" href="/education/front-end-web-development?where=atlanta">Front-End Web Development</a>,
 <a class="ProgramPanel__title___13Fzd" data-reactid="266" href="/education/javascript-development?where=atlanta">JavaScript Development</a>,
 <a class="ProgramPanel__title___13Fzd" data-reactid="272" href="/education/python-programming?where=atlanta">Pyth

In [12]:
programs = soup.find_all('a', {'class': 'ProgramPanel__title___13Fzd'})
topics = soup.find_all('p', {'class':'ProgramPanel__topic___3Jtm0'})

In [13]:
prog_list = []
for program in programs:
    prog_list.append(program.text)

In [14]:
prog_list

['Software Engineering Immersive',
 'Data Science Immersive',
 'User Experience Design Immersive',
 'Product Management',
 'Front-End Web Development',
 'JavaScript Development',
 'Python Programming',
 'React Development',
 'Data Analytics',
 'Data Science',
 'User Experience Design',
 'Visual Design',
 'Digital Marketing']

In [15]:
[topic.text for topic in topics]

['Coding',
 'Data',
 'Design',
 'Business',
 'Coding',
 'Coding',
 'Coding',
 'Coding',
 'Data',
 'Data',
 'Design',
 'Design',
 'Marketing']

## Scrape Each City for Basic Information

In [16]:
%%time
cities = ['Atlanta', 'Austin', 'Boston', 'Denver', 'Detroit', 'Houston', 'Los Angeles', 'Louisville', 'Miami',
          'Minneapolis', 'Nashville', 'New York City', 'Orlando', 'Phoenix', 'Raleigh', 'Salt Lake City', 'San Diego', 
          'San Francisco', 'Seattle', 'St Louis', 'Tampa', 'Washington DC', 'Toronto', 'Edinburgh', 'London', 'Manchester',
          'Paris', 'Singapore', 'Adelaide', 'Brisbane', 'Melbourne', 'Sydney']

USA = {'Atlanta': 'Georgia', 'Austin': 'Texas' , 'Boston': 'Massachusetts', 'Denver': 'Colorado', 'Detroit': 'Michigan',
       'Houston': 'Texas', 'Los Angeles': 'California', 'Louisville': 'Kentucky', 'Miami': 'Florida',
       'Minneapolis': 'Minnesota', 'Nashville': 'Tennessee', 'New York City': 'New York', 'Orlando': 'Florida', 
       'Phoenix': 'Arizona', 'Raleigh': 'North Carolina', 'Salt Lake City': 'Utah', 'San Diego': 'California', 
       'San Francisco': 'California', 'Seattle': 'Washington', 'St Louis': 'Missouri', 'Tampa': 'Florida', 'Washington DC': 'District of Columbia'}
Canada = ['Toronto']
Europe = {'Edinburgh': 'United Kingdom', 'London': 'United Kingdom', 'Manchester': 'United Kingdom', 'Paris': 'France'}
Asia = {'Tanjong Pagar':'Singapore'}
Australia = ['Adelaide', 'Brisbane', 'Melbourne', 'Sydney']


ga ={'city': None,
     'state': None,
     'country': None,
     'phone number': None,
     'email': None,
     'full-time courses': None,
     'part-time courses': None}
collection_dict = {ga['city']: ga}

count = 0

for city in cities:
    url = "https://generalassemb.ly/locations/" + city.replace(' ', '-').lower()
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")
# pulled in site______________________________________
    
    if city in Canada:
        country = 'Canada'
    elif city in Europe:
        country = Europe[city]
    elif city in list(Asia.values()):
        country = city
    elif city in Australia:
        country = 'Australia'
    else:
        country = 'USA'
    
    if city in USA:
        state = USA[city]
    else:
        state = 'N/A'
    
    
    full_time = []
    part_time = []
    programs = soup.find_all('a', {'class': 'ProgramPanel__title___13Fzd'})
    for course in programs:
        if 'immersive' in course.text.lower():
            full_time.append(course.text)
        else:
            part_time.append(course.text)
    
    
    if soup.find('a', {'class': "LocationMasthead__contact_phone___3ntPh"}) == None:
        phone = 'N/A'
    else:
        phone = soup.find('a', {'class': "LocationMasthead__contact_phone___3ntPh"}).text
        
    if soup.find('a', {'class': "LocationMasthead__contact_email___33lV7"}) == None:
        email = 'N/A'
    else:
        email = soup.find('a', {'class': "LocationMasthead__contact_email___33lV7"}).text
        
    if soup.find('address', {'class': 'Locations__campus_address___2iaHf'}) == None:
        address = 'N/A'
    else:
        address = soup.find('address', {'class': 'Locations__campus_address___2iaHf'}).text
        
    if city == 'Singapore':
        city = list(Asia.keys())[0]
    
    collection_dict[city] = {'city': city,
                             'state': state,
                             'country': country,
                             'address': address,
                             'phone number': phone,
                             'email': email,
                             'full-time courses': full_time,
                             'part-time courses': part_time}
    count += 1
    print(f'Finished scraping {city}: {count} out of {len(cities)}.')
    time.sleep(5)

collection_dict.pop(None)
print('Done!')


Finished scraping Atlanta: 1 out of 32.
Finished scraping Austin: 2 out of 32.
Finished scraping Boston: 3 out of 32.
Finished scraping Denver: 4 out of 32.
Finished scraping Detroit: 5 out of 32.
Finished scraping Houston: 6 out of 32.
Finished scraping Los Angeles: 7 out of 32.
Finished scraping Louisville: 8 out of 32.
Finished scraping Miami: 9 out of 32.
Finished scraping Minneapolis: 10 out of 32.
Finished scraping Nashville: 11 out of 32.
Finished scraping New York City: 12 out of 32.
Finished scraping Orlando: 13 out of 32.
Finished scraping Phoenix: 14 out of 32.
Finished scraping Raleigh: 15 out of 32.
Finished scraping Salt Lake City: 16 out of 32.
Finished scraping San Diego: 17 out of 32.
Finished scraping San Francisco: 18 out of 32.
Finished scraping Seattle: 19 out of 32.
Finished scraping St Louis: 20 out of 32.
Finished scraping Tampa: 21 out of 32.
Finished scraping Washington DC: 22 out of 32.
Finished scraping Toronto: 23 out of 32.
Finished scraping Edinburgh: 24 

In [17]:
collection_dict

{'Atlanta': {'city': 'Atlanta',
  'state': 'Georgia',
  'country': 'USA',
  'address': '675 Ponce De Leon Avenue NE, 2nd FloorAtlanta, GA 30308',
  'phone number': '+1 (404) 334-7858',
  'email': 'atlanta@generalassemb.ly',
  'full-time courses': ['Software Engineering Immersive',
   'Data Science Immersive',
   'User Experience Design Immersive'],
  'part-time courses': ['Product Management',
   'Front-End Web Development',
   'JavaScript Development',
   'Python Programming',
   'React Development',
   'Data Analytics',
   'Data Science',
   'User Experience Design',
   'Visual Design',
   'Digital Marketing']},
 'Austin': {'city': 'Austin',
  'state': 'Texas',
  'country': 'USA',
  'address': '600 Congress Avenue, 14th FloorAustin, TX 78701',
  'phone number': '+1 (512) 823-0359',
  'email': 'austin@generalassemb.ly',
  'full-time courses': ['Software Engineering Immersive',
   'Data Science Immersive',
   'User Experience Design Immersive'],
  'part-time courses': ['Product Managem

In [18]:
ga_information = pd.DataFrame.from_dict(collection_dict).T

In [19]:
# Already created
# ga_information.to_csv('./data/ga_scrape_info.csv', index = False)

## Formatting for YAML

In [20]:
ga_df = pd.read_csv('./data/ga_scrape_info.csv')

In [21]:
ga_df.head()

,city,state,country,address,phone number,email,full-time courses,part-time courses
0,Atlanta,Georgia,USA,"675 Ponce De Leon Avenue NE, 2nd FloorAtlanta,...",+1 (404) 334-7858,atlanta@generalassemb.ly,"['Software Engineering Immersive', 'Data Scien...","['Product Management', 'Front-End Web Developm..."
1,Austin,Texas,USA,"600 Congress Avenue, 14th FloorAustin, TX 78701",+1 (512) 823-0359,austin@generalassemb.ly,"['Software Engineering Immersive', 'Data Scien...","['Product Management', 'Front-End Web Developm..."
2,Boston,Massachusetts,USA,"125 Summer Street\r\n\r\n\r\n13th FloorBoston,...",+1 (617) 207-6245,boston@generalassemb.ly,"['Software Engineering Immersive', 'Data Scien...","['Product Management', 'Front-End Web Developm..."
3,Denver,Colorado,USA,"3858 Walnut StDenver, CO 80205",+1 (303) 963-9936,denver@generalassemb.ly,"['Software Engineering Immersive', 'Data Scien...","['Product Management', 'Front-End Web Developm..."
4,Detroit,Michigan,USA,NaN,+1 (917) 722-0237,detroit@generalassemb.ly,"['Software Engineering Immersive', 'Data Scien...","['Product Management', 'Front-End Web Developm..."


In [24]:
for ft in range(0, len(ga_df)):
    ga_df.loc[ft]['full-time courses'] = re.sub("[^,A-Za-z0-9]+", 
                                ' ', 
                                ga_df.loc[ft]['full-time courses']).strip().replace(' , ', ', ')

In [25]:
for pt in range(0, len(ga_df)):
    ga_df.loc[pt]['part-time courses'] = re.sub("[^,A-Za-z0-9]+", 
                                ' ', 
                                ga_df.loc[ft]['part-time courses']).strip().replace(' , ', ', ')

Parts of the following code have been commented out, so that the edited text file does not get overwritten.

In [26]:
%%time

CA_cities = []
TX_cities = []
FL_cities = []
Aus_cities = []
UK_cities = []
Can_cities = []
USA_states = []

for row in range(0, len(ga_df)):
    if ga_df.loc[row]['state'] == 'California':
        CA_cities.append(ga_df.loc[row]['city'])
    elif ga_df.loc[row]['state'] == 'Texas':
        TX_cities.append(ga_df.loc[row]['city'])
    elif ga_df.loc[row]['state'] == 'Florida':
        FL_cities.append(ga_df.loc[row]['city'])
    elif ga_df.loc[row]['country'] == 'Australia':
        Aus_cities.append(ga_df.loc[row]['city'])
    elif ga_df.loc[row]['country'] == 'United Kingdom':
        UK_cities.append(ga_df.loc[row]['city'])
    elif ga_df.loc[row]['country'] == 'Canada':
        Can_cities.append(ga_df.loc[row]['city'])
    elif ga_df.loc[row]['country'] == 'USA':
        USA_states.append(ga_df.loc[row]['state'])
    

california = f'- - What locations are available in California?\n'
california_rep = f'  - In California, General Assembly locations are available in {CA_cities}, and online.\n' 
texas = f'- - What locations are available in Texas?\n'
texas_rep = f'  - In Texas, General Assembly locations are available in {TX_cities}, and online.\n' 
florida = f'- - What locations are available in Florida?\n'
florida_rep = f'  - In Florida, General Assembly locations are available in {FL_cities}, and online.\n'

australia_loc = f'- - What locations are available in Australia?\n'
australia_loc_rep = f'  - In Australia, General Assembly locations are available in {Aus_cities}, and online.\n'
canada_loc = f'- - What locations are available in Canada?\n'
canada_loc_rep = f'  - In Canada, General Assembly the General Assembly location is in {Can_cities} and online.\n'
uk_loc = f'- - What locations are available in the United Kingdom?\n'
uk_loc_rep = f'  - In the United Kingdom, General Assembly locations are available in {UK_cities} and online.\n'
uk_loc2 = f'- - What locations are available in the UK?\n'
uk_loc_rep2 = f'  - In the United Kingdom, General Assembly locations are available in {UK_cities} and online.\n'
USA_loc = f'- - What locations are available in the US?\n'
USA_loc_rep = f'  - In the US, General Assembly locations are available in {USA_states} and online.\n'

# with open('./data/ga_info.txt', 'a') as files:
#     files.writelines(california)
#     files.writelines(california_rep)
#     files.writelines(texas)
#     files.writelines(texas_rep)
#     files.writelines(florida)
#     files.writelines(florida_rep)
#     files.writelines(australia_loc)
#     files.writelines(australia_loc_rep)
#     files.writelines(canada_loc)
#     files.writelines(canada_loc_rep)
#     files.writelines(uk_loc)
#     files.writelines(uk_loc_rep)
#     files.writelines(uk_loc2)
#     files.writelines(uk_loc_rep2)
#     files.writelines(USA_loc)
#     files.writelines(USA_loc_rep)

for city in range(0, len(ga_df)):
    state_locs = f"- - What General Assembly locations are available in {ga_df.loc[city]['state']}?\n"
    state_locs_rep = f"  - General Assembly-{ga_df.loc[city]['city']} is available in {ga_df.loc[city]['state']}.\n"
    state_locs2 = f"- - What locations are available in {ga_df.loc[city]['state']}?\n"
    state_locs_rep2 = f"  - General Assembly-{ga_df.loc[city]['city']} is available in {ga_df.loc[city]['state']}.\n"
    country_locs = f"- - What General Assembly locations are available in {ga_df.loc[city]['country']}?\n"
    country_locs_rep = f"  - {ga_df.loc[city]['city']} is available in {ga_df.loc[city]['country']}.\n"
    country_locs2 = f"- - What locations are available in {ga_df.loc[city]['country']}?\n"
    country_locs_rep2 = f"  - {ga_df.loc[city]['city']} is available in {ga_df.loc[city]['country']}.\n"
    address_locs = f"- - What is the address for General Assembly-{ga_df.loc[city]['city']}?\n"
    address_locs_rep = f"  - The address for General Assembly-{ga_df.loc[city]['city']} is {ga_df.loc[city]['address']}.\n"
    address_locs2 = f"- - Where is General Assembly in {ga_df.loc[city]['city']}?\n"
    address_locs_rep2 = f"  - The address for General Assembly-{ga_df.loc[city]['city']} is {ga_df.loc[city]['address']}.\n"
    ft_courses = f"- - What full-time courses are available in {ga_df.loc[city]['city']}?\n"
    ft_courses_rep = f"  - {ga_df.loc[city]['full-time courses']} are full-time courses available in {ga_df.loc[city]['city']}.\n"
    ft_courses2 = f"- - What full-time classes are available in {ga_df.loc[city]['city']}?\n"
    ft_courses_rep2 = f"  - {ga_df.loc[city]['full-time courses']} are full-time classes available in {ga_df.loc[city]['city']}.\n"
    pt_courses = f"- - What part-time courses are available in {ga_df.loc[city]['city']}?\n"
    pt_courses_rep = f"  - {ga_df.loc[city]['part-time courses']} are part-time courses available in {ga_df.loc[city]['city']}.\n"
    pt_courses2 = f"- - What part-time classes are available in {ga_df.loc[city]['city']}?\n"
    pt_courses_rep2 = f"  - {ga_df.loc[city]['part-time courses']} are part-time classes available in {ga_df.loc[city]['city']}.\n"
    phone_num = f"- - What is the phone number for General Assembly in {ga_df.loc[city]['city']}?\n"
    phone_num_rep = f"  - The phone number for General Assembly-{ga_df.loc[city]['city']} is {ga_df.loc[city]['phone number']}.\n"
    phone_num2 = f"- - What is the phone number for {ga_df.loc[city]['city']}?\n"
    phone_num_rep2 = f"  - The phone number for General Assembly-{ga_df.loc[city]['city']} is {ga_df.loc[city]['phone number']}.\n"
    email_add = f"- - What is the email for General Assembly in {ga_df.loc[city]['city']}?\n"
    email_add_rep = f"  - The email for General Assembly-{ga_df.loc[city]['city']} is {ga_df.loc[city]['email']}.\n"
    email_add2 = f"- - What is the email for {ga_df.loc[city]['city']}?\n"
    email_add_rep2 = f"  - The email for General Assembly-{ga_df.loc[city]['city']} is {ga_df.loc[city]['email']}.\n"

#     with open('./data/ga_info.txt', 'a') as files:
#         files.writelines(state_locs)
#         files.writelines(state_locs_rep)
#         files.writelines(state_locs2)
#         files.writelines(state_locs_rep2)
#         files.writelines(country_locs)
#         files.writelines(country_locs_rep)
#         files.writelines(country_locs2)
#         files.writelines(country_locs_rep2)
#         files.writelines(address_locs)
#         files.writelines(address_locs_rep)
#         files.writelines(address_locs2)
#         files.writelines(address_locs_rep2)
#         files.writelines(ft_courses)
#         files.writelines(ft_courses_rep)
#         files.writelines(ft_courses2)
#         files.writelines(ft_courses_rep2)
#         files.writelines(pt_courses)
#         files.writelines(pt_courses_rep)
#         files.writelines(pt_courses2)
#         files.writelines(pt_courses_rep2)
#         files.writelines(phone_num)
#         files.writelines(phone_num_rep)
#         files.writelines(phone_num2)
#         files.writelines(phone_num_rep2)
#         files.writelines(email_add)
#         files.writelines(email_add_rep)
#         files.writelines(email_add2)
#         files.writelines(email_add_rep2)
    


CPU times: user 210 ms, sys: 6.24 ms, total: 216 ms
Wall time: 223 ms


In [41]:
# Commented out: Already completed

# with open('./data/ga_info.txt') as f:
#     data = yaml.load(f)
#     yml_file = open('./data/ga_info.yml', 'w')
#     yaml.dump(data, yml_file, default_flow_style=False)

# Scrape Course Information

In [28]:
url2 = "https://generalassemb.ly/education/software-engineering-immersive"

response2 = requests.get(url2)

print(response2.status_code)
time.sleep(2)

200


In [29]:
html2 = response2.text
soup2 = BeautifulSoup(html2, "lxml")

In [30]:
soup2.find_all('p', {'class': 'ga-type-body-sm'})[0].text

"Gain a software engineer's skill set from leading practitioners. Top-notch instruction at GA means industry-validated curriculum, hours of guided coding practice, and real-world projects."

In [31]:
soup2.find_all('p', {'class': 'ga-type-body-sm'})[4].text

'Hone fluency in multiple frameworks and stacks. Apply essential object-oriented programming concepts, and leverage JavaScript libraries like React. Incorporate functionality from third-party APIs and add complexity and nuance with Python, Django, and more.'

In [32]:
%%time
ga_courses = ['Software Engineering Immersive',
                'User Experience Design Immersive',
                'Data Science Immersive',
                'Digital Marketing Immersive',
                'Software Engineering Immersive Remote Flex',
                'Python Programming',
                'User Experience Design',
                'Data Analytics',
                'Digital Marketing',
                'Data Science',
                'Product Management',
                'Visual Design',
                'Front End Web Development',
                'JavaScript Development',
                'React Development']

count = 0
course_dict = {}
for course in ga_courses:
    url = "https://generalassemb.ly/education/" + course.replace(' ', '-').lower()
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")

    if 'immersive' in course.lower():
        #full-time immersive courses
        description1 = soup.find_all('p', {'class': 'ga-type-body-sm'})[3].text
        description2 = soup.find_all('p', {'class': 'ga-type-body-sm'})[4].text
        course_dict[course] = description1 + ' ' + description2
    else: 
        #part-time immersive courses
        description3 = soup.find_all('p', {'class': 'ga-type-body-sm'})[0].text
        description4 = soup.find_all('p', {'class': 'ga-type-body-sm'})[1].text
        course_dict[course] = description3 + ' ' + description4
    
    count += 1
    print(f'Finished scraping {course}: {count} out of {len(ga_courses)}.')
    time.sleep(5)
    
print('Done!')

Finished scraping Software Engineering Immersive: 1 out of 15.
Finished scraping User Experience Design Immersive: 2 out of 15.
Finished scraping Data Science Immersive: 3 out of 15.
Finished scraping Digital Marketing Immersive: 4 out of 15.
Finished scraping Software Engineering Immersive Remote Flex: 5 out of 15.
Finished scraping Python Programming: 6 out of 15.
Finished scraping User Experience Design: 7 out of 15.
Finished scraping Data Analytics: 8 out of 15.
Finished scraping Digital Marketing: 9 out of 15.
Finished scraping Data Science: 10 out of 15.
Finished scraping Product Management: 11 out of 15.
Finished scraping Visual Design: 12 out of 15.
Finished scraping Front End Web Development: 13 out of 15.
Finished scraping JavaScript Development: 14 out of 15.
Finished scraping React Development: 15 out of 15.
Done!
CPU times: user 1.62 s, sys: 57 ms, total: 1.67 s
Wall time: 3min 59s


In [33]:
course_dict

{'Software Engineering Immersive': 'Developed with guidance from our software engineering advisory board, our curriculum is primed for industry relevance. Learn the core skills to build full-stack web applications on the job: computer science fundamentals, collaboration tools, and the programming languages modern employers demand. Hone fluency in multiple frameworks and stacks. Apply essential object-oriented programming concepts, and leverage JavaScript libraries like React. Incorporate functionality from third-party APIs and add complexity and nuance with Python, Django, and more.',
 'User Experience Design Immersive': 'Developed with guidance from our User Experience Design Standards Board — a group of executives from companies like Tigerspike and WarnerMedia — our upgraded UX bootcamp curriculum is primed for industry relevance. Get hands-on experience applying essential tactics like wireframing, prototyping, user research, and usability testing. Dive deep into visual and user inte

In [34]:
len(course_dict)

15

In [35]:
course_df = pd.DataFrame.from_dict(course_dict, orient='index', columns = ['description'])

In [36]:
course_df.reset_index(inplace = True)
course_df.rename(columns = {'index':'course'}, inplace = True)

In [37]:
# course_df.to_csv('./data/ga_course_info.csv', index = False)

## Formatting for YAML

In [38]:
course_df.loc[0]['description']

'Developed with guidance from our software engineering advisory board, our curriculum is primed for industry relevance. Learn the core skills to build full-stack web applications on the job: computer science fundamentals, collaboration tools, and the programming languages modern employers demand. Hone fluency in multiple frameworks and stacks. Apply essential object-oriented programming concepts, and leverage JavaScript libraries like React. Incorporate functionality from third-party APIs and add complexity and nuance with Python, Django, and more.'

In [39]:
for row in range(0, len(course_df)):
    course_q = f"- - What is {course_df.loc[row]['course']}?\n"
    course_a = f"  - {course_df.loc[row]['description']}\n"
    
#     with open('./data/ga_course_info.txt', 'a') as files:
#         files.writelines(course_q)
#         files.writelines(course_a)

In [40]:
# %%time
# with open('./data/ga_course_info.txt') as f:
#     data = yaml.load(f)
#     yml_file = open('./data/ga_course_info.yml', 'w')
#     yaml.dump(data, yml_file, default_flow_style=False)